In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_stains.zip to final_stains.zip


In [ ]:
!unzip -o final_stains.zip -d /content/

Archive:  final_stains.zip
   creating: /content/final_stains/
   creating: /content/final_stains/blood/
  inflating: /content/final_stains/blood/0.jpg  
  inflating: /content/final_stains/blood/1.jpg  
  inflating: /content/final_stains/blood/10.jpg  
  inflating: /content/final_stains/blood/11.jpg  
  inflating: /content/final_stains/blood/12.jpg  
  inflating: /content/final_stains/blood/13.jpg  
  inflating: /content/final_stains/blood/14.jpg  
  inflating: /content/final_stains/blood/15.jpg  
  inflating: /content/final_stains/blood/16.jpg  
  inflating: /content/final_stains/blood/17.jpg  
  inflating: /content/final_stains/blood/18.jpg  
  inflating: /content/final_stains/blood/19.jpg  
  inflating: /content/final_stains/blood/2.jpg  
  inflating: /content/final_stains/blood/20.jpg  
  inflating: /content/final_stains/blood/21.jpg  
  inflating: /content/final_stains/blood/22.jpg  
  inflating: /content/final_stains/blood/23.jpg  
  inflating: /content/final_stains/blood/24.jpg 

In [7]:
!pip install ultralytics pillow tqdm

import os, random, shutil
from glob import glob
from tqdm import tqdm

from ultralytics import YOLO

BASE_DIR = "/content/final_stains"
SPLIT_DIR = "/content/stains_split"

os.makedirs(SPLIT_DIR, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("클래스 폴더:", os.listdir(BASE_DIR))


print("\n[train/val split 시작]\n")

train_root = os.path.join(SPLIT_DIR, "train")
val_root   = os.path.join(SPLIT_DIR, "val")

os.makedirs(train_root, exist_ok=True)
os.makedirs(val_root, exist_ok=True)

for cls in os.listdir(BASE_DIR):
    cls_path = os.path.join(BASE_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    exts = ("*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG")
    files = []
    for e in exts:
        files.extend(glob(os.path.join(cls_path, e)))

    if len(files) == 0:
        print(f"⚠️ {cls} 폴더에 이미지가 없습니다. 건너뜀.")
        continue

    random.shuffle(files)

    cut = int(len(files) * 0.8)
    train_files = files[:cut]
    val_files   = files[cut:]

    train_dir = os.path.join(train_root, cls)
    val_dir   = os.path.join(val_root, cls)
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    for f in train_files:
        shutil.copy(f, os.path.join(train_dir, os.path.basename(f)))
    for f in val_files:
        shutil.copy(f, os.path.join(val_dir, os.path.basename(f)))

    print(f"✅ {cls}: train {len(train_files)}장, val {len(val_files)}장")

print("\n✅ train/val split 전체 완료!")
!ls -R /content/stains_split


import yaml

classes = sorted(
    [d for d in os.listdir(train_root) if os.path.isdir(os.path.join(train_root, d))]
)

data_yaml = {
    "path": SPLIT_DIR,
    "train": "train",
    "val": "val",
    "names": classes,
}

yaml_path = os.path.join(SPLIT_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f)

print("\n[생성된 data.yaml]")
print(open(yaml_path).read())




model = YOLO("yolov8n-cls.pt")

model.train(
    data="/content/stains_split",
    epochs=30,
    imgsz=224,
    batch=32,

)
best_path = "/content/runs/classify/train/weights/best.pt"
print(f"\n📌 최종 모델 위치: {best_path}")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
BASE_DIR: /content/final_stains


FileNotFoundError: [Errno 2] No such file or directory: '/content/final_stains'

In [ ]:
from google.colab import files
import glob

# YOLO가 만든 weights 폴더 전체에서 best.pt 찾기
best_paths = glob.glob("/content/runs/classify/**/weights/best*.pt", recursive=True)

if len(best_paths) == 0:
    print("❌ best.pt 파일을 찾을 수 없습니다.")
else:
    best_path = best_paths[0]
    print("📌 찾은 best.pt:", best_path)
    files.download(best_path)

In [3]:
import os
from glob import glob
from ultralytics import YOLO
from IPython.display import Image, display
import pandas as pd

best_path = "/content/runs/classify/train/weights/best.pt"
SPLIT_DIR = "/content/stains_split"
results_dir = "/content/runs/classify/train"

print("="*60)
print("="*60)


confusion_matrix_path = f"{results_dir}/confusion_matrix_normalized.png"

if os.path.exists(confusion_matrix_path):
    display(Image(filename=confusion_matrix_path))
    print("✅ Confusion Matrix를 확인하세요!")
    print("   - 대각선: 정확히 맞춘 것")
    print("   - 대각선 외: 헷갈린 것")
else:
    print("⚠️ Confusion Matrix 파일이 없습니다.")



results_png = f"{results_dir}/results.png"

if os.path.exists(results_png):
    display(Image(filename=results_png))
else:
    print("⚠️ 학습 곡선 파일이 없습니다.")


print("\n" + "="*60)
print("🎯 정확도 상세 정보")
print("="*60)

results_csv = f"{results_dir}/results.csv"

if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)


    print("\n📊 마지막 5 Epoch 결과:")
    print(df[['epoch', 'train/loss', 'metrics/accuracy_top1', 'metrics/accuracy_top5']].tail(5))

    best_top1 = df['metrics/accuracy_top1'].max()
    best_top5 = df['metrics/accuracy_top5'].max()
    best_epoch = df['metrics/accuracy_top1'].idxmax() + 1

    print(f"\n✨ 최고 성능:")
    print(f"   🥇 Top-1 정확도: {best_top1:.4f} ({best_top1*100:.2f}%)")
    print(f"   🥇 Top-5 정확도: {best_top5:.4f} ({best_top5*100:.2f}%)")
    print(f"   📍 최고 Epoch: {best_epoch}")
else:
    print("⚠️ results.csv 파일이 없습니다.")

print("\n" + "="*60)
print("🧪 검증 데이터로 최종 평가")
print("="*60)

if os.path.exists(best_path):
    best_model = YOLO(best_path)

    val_results = best_model.val(
        data=SPLIT_DIR,
        split="val"
    )

    print(f"\n✅ 검증 세트 정확도:")
    print(f"   🎯 Top-1: {val_results.top1:.4f} ({val_results.top1*100:.2f}%)")
    print(f"   🎯 Top-5: {val_results.top5:.4f} ({val_results.top5*100:.2f}%)")
else:
    print(f"❌ 모델 파일을 찾을 수 없습니다: {best_path}")
    print("\n모델 위치 찾기:")
    !find /content -name "best.pt" 2>/dev/null


print("\n" + "="*60)
print("="*60)

if os.path.exists(best_path):
    val_dir = f"{SPLIT_DIR}/val"

    if os.path.exists(val_dir):
        print(f"\n{'실제 클래스':<15} | {'예측 클래스':<15} | {'신뢰도':<8} | 결과")
        print("-" * 60)

        for cls in sorted(os.listdir(val_dir)):
            cls_path = os.path.join(val_dir, cls)
            if not os.path.isdir(cls_path):
                continue

            files = glob(os.path.join(cls_path, "*.jpg")) + \
                    glob(os.path.join(cls_path, "*.png"))

            if len(files) > 0:
                sample = files[0]
                result = best_model(sample)[0]
                pred_cls = result.names[result.probs.top1]
                conf = float(result.probs.top1conf)

                status = "✅ 정답" if pred_cls == cls else "❌ 오답"
                print(f"{cls:<15} | {pred_cls:<15} | {conf:>6.1%} | {status}")
    else:
        print(f"⚠️ 검증 폴더가 없습니다: {val_dir}")

print("\n" + "="*60)
print("="*60)

ModuleNotFoundError: No module named 'ultralytics'